In [ ]:
%%capture
# mount your google drive to this notebook
from google.colab import drive
drive.flush_and_unmount()
drive.mount("mnt", force_remount=True)

In [ ]:
%%capture
!pip install adapters transformers datasets evaluate

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2ForTokenClassification, GPT2ForQuestionAnswering
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

In [ ]:
# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Assign the eos_token as padding

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from adapters import AdapterConfig, AutoAdapterModel

def load_model_for_task(task_name, adapter_type='full_finetune', model_type='sequence_classification'):
    """
    Load GPT-2 model for specific task and fine-tuning type.
    Args:
        task_name (str): The name of the task (e.g., 'sst-2', 'conll', 'squad', 'writingprompts').
        adapter_type (str): Fine-tuning type: 'full_finetune', 'adapter_finetune', 'adapter_stitching'.
        model_type (str): The model type: 'sequence_classification', 'token_classification', 'question_answering'.
    Returns:
        model: Loaded GPT-2 model for the task.
    """
    if model_type == 'sequence_classification':
        model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
    elif model_type == 'token_classification':
        model = GPT2ForTokenClassification.from_pretrained('gpt2', num_labels=9)
    elif model_type == 'question_answering':
        model = GPT2ForQuestionAnswering.from_pretrained('gpt2')

    # Load or create adapters based on the fine-tuning type
    if adapter_type == 'adapter_finetune':
        config = AdapterConfig.load("pfeiffer")
        model.add_adapter(f"{task_name}_adapter", config=config)
        model.train_adapter(f"{task_name}_adapter")
    elif adapter_type == 'adapter_stitching':
        # Stitch adapters from other tasks
        for other_task in ['sst-2', 'conll', 'squad', 'writingprompts']:
            if other_task != task_name:
                model.load_adapter(f"{other_task}_adapter")
        model.train_adapter([f"{task_name}_adapter"])

    return model

In [ ]:
def load_task_dataset(task_name):
    """
    Load and preprocess the dataset for the specified task.
    Args:
        task_name (str): The name of the task (e.g., 'sst-2', 'conll', 'squad', 'writingprompts').
    Returns:
        dataset: The preprocessed dataset for the task.
    """
    if task_name == 'sst-2':
        dataset = load_dataset('glue', 'sst2')
        metric = evaluate.load('accuracy')
    elif task_name == 'conll':
        dataset = load_dataset('conll2003')
        metric = evaluate.load('f1')
    elif task_name == 'squad':
        dataset = load_dataset('squad_v2')
        metric = evaluate.load('squad_v2')
    elif task_name == 'writingprompts':
        dataset = load_dataset('writing_prompts')
        metric = None  # Perplexity or human evaluation may be used for text generation

    return dataset, metric

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

# Tokenizer function to add input_ids and attention_mask
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

def train_model(task_name, adapter_type='full_finetune', model_type='sequence_classification'):
    """
    Train GPT-2 model on specific task and fine-tuning type.
    Args:
        task_name (str): The name of the task (e.g., 'sst2').
        adapter_type (str): The type of fine-tuning (full_finetune, adapter_finetune, adapter_stitching).
        model_type (str): The type of model to load (sequence_classification, etc.).
    """
    # Load dataset
    dataset = load_dataset("glue", task_name)

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Set padding token
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

    # Tokenize dataset
    tokenized_datasets = dataset.map(lambda examples: tokenize_function(examples, tokenizer), batched=True)

    # Load GPT-2 model for sequence classification with two labels (SST-2: positive or negative sentiment)
    if model_type == 'sequence_classification':
        model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

        # GPT-2 needs to have a padding token to work with attention masks
        model.config.pad_token_id = model.config.eos_token_id

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                 # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        eval_strategy="steps",           # Updated from evaluation_strategy
    )

    # Define the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()

In [ ]:
def evaluate_model(task_name, adapter_type='full_finetune', model_type='sequence_classification'):
    """
    Evaluate the GPT-2 model on the validation dataset.
    Args:
        task_name (str): Task name (e.g., 'sst-2', 'conll', 'squad', 'writingprompts').
        adapter_type (str): Type of fine-tuning (full, adapter, stitching).
        model_type (str): Model type (sequence classification, token classification, etc.).
    """
    model = load_model_for_task(task_name, adapter_type, model_type)
    dataset, metric = load_task_dataset(task_name)

    trainer = Trainer(
        model=model,
        args=TrainingArguments(output_dir=f"mnt/MyDrive/Results/results/{task_name}_eval"),
        eval_dataset=dataset['validation'],
        tokenizer=tokenizer
    )

    results = trainer.evaluate()
    print(results)


In [ ]:
# Train model for SST-2 using full fine-tuning
train_model('sst2', 'full_finetune', 'sequence_classification')

# Train model for SST-2 using adapter fine-tuning
train_model('sst2', 'adapter_finetune', 'sequence_classification')

# Evaluate the model for SST-2
evaluate_model('sst2', 'full_finetune', 'sequence_classification')

# Adapter stitching for SST-2
train_model('sst2', 'adapter_stitching', 'sequence_classification')

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
10,2.562800,2.513299
20,2.629700,2.201130
30,2.414200,1.723064
40,1.657700,1.008083
50,1.168900,0.785554
60,0.856500,0.744980
70,0.716900,0.680710
80,0.768000,0.701244
90,0.801900,0.726485
100,0.720600,0.631483
